## 네 번째 모델링 : 1~3번 째 모델링에서 뽑은 특성중요도에서 낮은 값을 가지는 컬럼을 제외한 모델링

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv('./train_new.csv')
train.drop(['ID','근로기간','주택소유상태','총연체금액','연체계좌수'], axis=1, inplace=True)

In [2]:
train.head()

,대출금액,대출기간,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,대출등급
0,12480000,3,72000000,18.90,15,부채 통합,0,0,0.0,C
1,14400000,5,130800000,22.33,21,주택 개선,0,373572,234060.0,B
2,12000000,3,96000000,8.60,14,부채 통합,0,928644,151944.0,A
3,14400000,3,132000000,15.09,15,부채 통합,0,325824,153108.0,C
4,4800000,3,84000000,13.78,30,휴가,0,240216,55428.0,A


In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

train['대출목적'] = le.fit_transform(train['대출목적'])
train['대출등급'] = le.fit_transform(train['대출등급'])

In [10]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_ss = ss.fit_transform(X)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_ss, y, test_size=0.3, stratify=y, random_state=42)

In [12]:
def grid_search(model, params, random=False):
    clf = model
    if not random:
        grid = GridSearchCV(clf, params,
                                scoring='f1_macro', cv=5,
                                n_jobs=-1)
    else:
        grid = RandomizedSearchCV(clf, params, n_iter=10,
                                scoring='f1_macro', cv=5,
                                n_jobs=-1, random_state=42)
        
    grid.fit(X_train, y_train)
    
    
    best_model = grid.best_estimator_
    
    best_params = grid.best_params_
    print("최상의 매개변수:", best_params)
    
    best_score = grid.best_score_
    print("훈련 점수: {:.3f}".format(best_score))
    
    y_pred = best_model.predict(X_val)
    macro_f1_val = f1_score(y_val, y_pred, average='macro')
    print('테스트 세트 점수: {:.3f}'.format(macro_f1_val))

In [9]:
params = {'min_samples_leaf':[18,19,20,21,22],
          'min_impurity_decrease':[0.0],
          'max_features':['auto',0.6,0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.69,0.70],
          'max_depth':[None,11,12,13,14,15,16,17,18],
          'class_weight' : [None, 'balanced']}

grid_search(DecisionTreeClassifier(random_state=42), params, random=True)

최상의 매개변수: {'min_samples_leaf': 19, 'min_impurity_decrease': 0.0, 'max_features': 0.68, 'max_depth': 17, 'class_weight': None}
훈련 점수: 0.691
테스트 세트 점수: 0.687


In [13]:
params = {'min_samples_leaf':[1,2,3,4,5,6,7,8,9],
          'min_impurity_decrease':[0.0],
          'max_features':[0.65,0.66,0.67,0.68,0.69,0.7,0.71,0.72,0.73,0.74,0.75],
          'max_depth':[90,91,92,93,94,95,96,97,98,99,100],
          'n_estimators' : [750,760,770,780,790,800,810,820,830,840,850]}

grid_search(RandomForestClassifier(n_jobs=-1, random_state=42), params, random=True)

최상의 매개변수: {'n_estimators': 810, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0, 'max_features': 0.73, 'max_depth': 95}
훈련 점수: 0.794
테스트 세트 점수: 0.800


In [10]:
params = {'n_estimators' : [130,150,170],
          'learning_rate' : [0.18,0.2,0.22],
          'max_depth' : [33,34,35,36,37],
          'objective' : ['multi:softmax']}

grid_search(XGBClassifier(random_state=42, n_jobs=-1), params, random=True)

최상의 매개변수: {'objective': 'multi:softmax', 'n_estimators': 130, 'max_depth': 37, 'learning_rate': 0.18}
훈련 점수: 0.788
테스트 세트 점수: 0.795


욕심내서 기본 학습기 gbtree, gblinear, dart 3개 비교해보려고 튜닝 돌렸는데 13시간 돌려도 안나와서 포기.......................... 원래 깔아놨던 파라미터만 튜닝

⑥ LGBMClassifier 하이퍼파라미터 튜닝

In [11]:
params = {'n_estimators' : [340,350,360],
          'learning_rate' : [0.05,0.06,0.07,0.08,0.09],
          'max_depth' : [8,9,10,11,12],
          'num_leaves' : [43,45,47]}

grid_search(LGBMClassifier(objective='multiclass', random_state=42, n_jobs=-1), params, random=True)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002590 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1404
[LightGBM] [Info] Number of data points in the train set: 63435, number of used features: 9
[LightGBM] [Info] Start training from score -1.744243
[LightGBM] [Info] Start training from score -1.208106
[LightGBM] [Info] Start training from score -1.248814
[LightGBM] [Info] Start training from score -1.982449
[LightGBM] [Info] Start training from score -2.564256
[LightGBM] [Info] Start training from score -3.885346
[LightGBM] [Info] Start training from score -5.433754
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

### StratifiedKFold를 이용한 스코어 산정

In [14]:
X_ss = pd.DataFrame(X_ss, columns=X.columns)

from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

f1_macro_scores = []

def skf_score(model):
    for train_idx, valid_idx in skf.split(X_ss, y):
        X_train = X_ss.iloc[train_idx]
        X_val = y.iloc[train_idx]

        y_train = X_ss.iloc[valid_idx]
        y_val = y.iloc[valid_idx]

        model.fit(X_train, X_val)

        pred = model.predict(y_train)

        f1_macro = f1_score(y_val, pred, average='macro')
        f1_macro_scores.append(f1_macro)
    
    average_f1_macro = np.mean(f1_macro_scores)

    print("Average F1-macro score:", average_f1_macro)
    try:
        if model.feature_importances_.any():
            feature_importances = model.feature_importances_
            print("\n",'-'*10,'특성중요도','-'*10)
            for feature, importance in zip(X_ss.columns, feature_importances):
                print(f"{feature}: {importance}")
    except:
        None

In [13]:
skf_score(DecisionTreeClassifier(min_samples_leaf=19, min_impurity_decrease= 0, max_features=0.68,
                                 max_depth=17, class_weight=None, random_state=42))

Average F1-macro score: 0.725750286030886

 ---------- 특성중요도 ----------
대출금액: 0.043565903951824185
대출기간: 0.04904212766776253
연간소득: 0.018353171243394077
부채_대비_소득_비율: 0.004422153852233158
총계좌수: 0.0024230350508048634
대출목적: 0.0022002837516323834
최근_2년간_연체_횟수: 0.0009116126919420291
총상환원금: 0.4558304869497931
총상환이자: 0.4232512248406137


In [15]:
skf_score(RandomForestClassifier(n_estimators=810, min_samples_leaf=3, min_impurity_decrease=0, 
                                 max_features=0.73, max_depth=95, random_state=42, n_jobs=-1))

Average F1-macro score: 0.8188658057990355

 ---------- 특성중요도 ----------
대출금액: 0.07114289441263649
대출기간: 0.03705193802839339
연간소득: 0.032307919356093805
부채_대비_소득_비율: 0.030466462533148504
총계좌수: 0.022397910825849365
대출목적: 0.007261120898961906
최근_2년간_연체_횟수: 0.0037971483633356226
총상환원금: 0.4238118682188787
총상환이자: 0.37176273736270227


In [17]:
skf_score(XGBClassifier(objective='multi:softmax', n_estimators=130, max_depth=37, 
          learning_rate=0.18, n_jobs=-1, random_state=42))

Average F1-macro score: 0.8093017599586311

 ---------- 특성중요도 ----------
대출금액: 0.028552057221531868
대출기간: 0.5786954164505005
연간소득: 0.01842695288360119
부채_대비_소득_비율: 0.013733377680182457
총계좌수: 0.013640983030200005
대출목적: 0.018290190026164055
최근_2년간_연체_횟수: 0.016773225739598274
총상환원금: 0.1495564728975296
총상환이자: 0.16233129799365997


In [16]:
skf_score(LGBMClassifier(objective='multiclass', num_leaves=45, n_estimators=360, max_depth=11, 
               learning_rate=0.09, n_jobs=-1, random_state=42))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.088524 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1408
[LightGBM] [Info] Number of data points in the train set: 81559, number of used features: 9
[LightGBM] [Info] Start training from score -1.744289
[LightGBM] [Info] Start training from score -1.208056
[LightGBM] [Info] Start training from score -1.248847
[LightGBM] [Info] Start training from score -1.982382
[LightGBM] [Info] Start training from score -2.564275
[LightGBM] [Info] Start training from score -3.885514
[LightGBM] [Info] Start training from score -5.434151
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001124 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not e

model|k-Fold|Sk-Fold
-|-|-
DecisionTree Classifier|0.687|0.725750286030886
RandomForest Classifier|0.800|0.8188658057990355 --> 최고 성능
XGBoost Classifier|0.795|0.8093017599586311
Light GBM Classifier|0.775|0.8085671220859905